<a href="https://colab.research.google.com/github/anupampani123/LyricGenerator/blob/main/JeffBuckleySongGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np


In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 9.3MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
     |████████████████████████████████| 2.9MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b64e299a2a07f34e8b3b0ada9e3d68ea764c5e49a9c9a5977cd830226164753e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [ ]:
tokenizer = Tokenizer()
data = open('lyrics_scraped.txt',encoding="utf8").read()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
# tokenizer2 = Tokenizer()
# tokenizer2.fit_on_texts(corpus)
# total_words = len(tokenizer2.word_index) + 1


In [ ]:
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights)

In [ ]:
import pandas as pd
corpus1=pd.DataFrame(corpus,columns=['lyric'])

In [ ]:
tokenized=[]

In [ ]:
# for i in range(0,len(corpus)):
#   tokenized.append(tokenizer.encode(corpus[i]))
  

In [ ]:
# max_len = 0
# for i in range(0,len(tokenized)):
#     if len(tokenized[i]) > max_len:
#         max_len = len(tokenized[i])

# padded = np.array([i + [0]*(max_len-len(tokenized[i])) for i in range(0,len(tokenized))])

TypeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

[101, 1037, 1045, 2293, 102]

In [ ]:
input_sequences = []
for line in corpus:
 token_list = tokenizer.texts_to_sequences([line])[0]
 for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  input_sequences.append(n_gram_sequence)
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
# input_sequences=tokenized
# max_sequence_len = max([len(x) for x in input_sequences])
# input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# predictors= input_sequences[:,:-2]

# label=input_sequences[:,-3]

In [ ]:
# label=input_sequences[:,-3]
# total_words=30522
# label=ku.to_categorical(label, num_classes=total_words)

In [ ]:
# label

array([1001,  101, 3398, ..., 1012,  101,  101], dtype=int32)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 100)           3052200   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 30, 300)           301200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 300)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_4 (Dense)              (None, 15261)             1541361   
_________________________________________________________________
dense_5 (Dense)              (None, 30522)             465826764 
Total params: 470,881,925
Trainable params: 470,881,925
Non-trainable params: 0
________________________________________

In [ ]:
history = model.fit(predictors, label, epochs=10, verbose=1)

Epoch 1/10
67/67 [==============================] - 13s 147ms/step - loss: 8.3451 - accuracy: 0.1516
Epoch 2/10
67/67 [==============================] - 10s 146ms/step - loss: 4.5153 - accuracy: 0.2348
Epoch 3/10
67/67 [==============================] - 10s 146ms/step - loss: 4.0801 - accuracy: 0.2541
Epoch 4/10
67/67 [==============================] - 10s 147ms/step - loss: 4.0061 - accuracy: 0.2526
Epoch 5/10
67/67 [==============================] - 10s 146ms/step - loss: 3.6802 - accuracy: 0.2867
Epoch 6/10
67/67 [==============================] - 10s 146ms/step - loss: 3.5634 - accuracy: 0.2898
Epoch 7/10
67/67 [==============================] - 10s 147ms/step - loss: 3.3878 - accuracy: 0.3182
Epoch 8/10
67/67 [==============================] - 10s 146ms/step - loss: 3.2500 - accuracy: 0.3290
Epoch 9/10
67/67 [==============================] - 10s 145ms/step - loss: 3.0497 - accuracy: 0.3623
Epoch 10/10
67/67 [==============================] - 10s 146ms/step - loss: 2.8507 - accura

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
model.save("buckleysong.h5")

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model("buckleysong.h5")

# Show the model architecture
new_model.summary()

OSError: ignored

In [ ]:
seed_text = "I am the highway"
next_words = 8
  
for _ in range(next_words):

 token_list = tokenizer.texts_to_sequences([seed_text])[0]
 token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  # token_list=tokenizer.encode(seed_text,add_special_tokens=True)
  # token_list = (pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre'))
  
  predicted = model.predict_classes(token_list, verbose=0)
  output_word = ""
 for word, index in tokenizer.word_index.items():
  if index == predicted:
   output_word = word
   break
  seed_text += " " + tokenizer.decode(predicted) 
print(seed_text)

I am the highway another another another her her her her her


In [ ]:
token_list=tokenizer.encode("I am the highway in lose",add_special_tokens=True)

In [ ]:
token_list

[101, 1045, 2572, 1996, 3307, 1999, 4558, 102]

In [ ]:
tokenizer.decode(predict1)

'lose'